<a href="https://colab.research.google.com/github/AlexFaida/projects_Python/blob/Web-scraping/%D0%9F%D0%B0%D1%80%D1%81%D0%B5%D1%80_%D0%B8%D0%BD%D0%B2%D0%B5%D1%81%D1%82_%D0%BF%D1%80%D0%BE%D0%B5%D0%BA%D1%82%D0%BE%D0%B2_%D0%BD%D0%BE%D0%B2%D0%BE%D1%81%D1%82%D0%B5%D0%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ссылки на используемые сайты

**Новости:**
- **Gazeta.uz**: [Ссылка](https://www.gazeta.uz/ru/archive)
- **Kun.uz**: [Ссылка](https://kun.uz/ru/news/list)
- **Sputnik Uzbekistan (Economy)**: [Ссылка](https://uz.sputniknews.ru/economy/)
- **Dzen.ru (Uzbekistan)**: [Ссылка](https://dzen.ru/news/region/uzbekistan)
- **BBGL.ru (Uzbekistan)**: [Ссылка](https://bbgl.ru/news?hashtag_place=%D0%A3%D0%B7%D0%B1%D0%B5%D0%BA%D0%B8%D1%81%D1%82%D0%B0%D0%BD)
- **Uzbekistonmet.uz (Economic)**: [Ссылка](https://www.uzbekistonmet.uz/ru/lists/category/5)

**Инвестирование:**
- **Invest.gov.uz (Ready Invest Projects)**: [Ссылка](https://invest.gov.uz/ru/investor-taxonomy/ready-invest-projects/)
- **Invest-in-uzbekistan.org (Enterprises Projects)**: [Ссылка](https://invest-in-uzbekistan.org/enterprises-projects/)

# Необходимые библиотеки

In [ ]:
# Необходимо для работы в google colab
## Можно не устанавливать, если код запущен локально
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver

In [ ]:
#необходимые библиотеки
!pip install selenium
!pip install chromedriver-autoinstaller
!pip install xlsxwriter

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import chromedriver_autoinstaller
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
import re
from urllib.parse import quote
import numpy as np
from datetime import datetime, timedelta

# Блок новостей

Запросы без указаний осуществляются по ключевым словам:

*   инвест
*   разраб
*   строит
*   модерн

In [ ]:
def kun_uz(queries):

  titles, links, dates = [], [], []
  pages = 2  # количество страниц новостей, которое необходимо спарсить. Большинство запросов запросов умещаются в 1 страницу.
  links_of_queries = [[f' https://kun.uz/ru/news/search?q={quote(query)}&page={page}' for query in queries] for page in range(1,pages+1)]
  links_of_queries_array = np.array(links_of_queries)
  links_of_queries_flat = links_of_queries_array.flatten()

  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  chromedriver_autoinstaller.install()
  driver = webdriver.Chrome(options=chrome_options)
  driver.maximize_window()

  wait = WebDriverWait(driver, 10)

  current_time = datetime.now()
  current_date = current_time.strftime("%d.%m.%Y")


  for link_of_query in links_of_queries_flat:

    driver.get(link_of_query)
    wait = WebDriverWait(driver, 10)

    try:
        news_elements = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "news__title")))
        date_elements = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="news-list"]/div/div/div')))

    except Exception as e:
        print(f'Произошла ошибка с - {e} по ссылке {link_of_query}')

    try:
      for single_news in news_elements:

        titles.append(single_news.text)
        links.append(single_news.get_attribute("href"))

      for single_news in date_elements:

        dates.append(single_news.text)

    except Exception as e:
        print(f'Произошла ошибка с - {e} по ссылке {link_of_query}')

  driver.quit()

  dates_f = []
  try:
    for date in dates:
        if '/' in date:
            date_filt = ''.join(date.split(' / ')[1])
            dates_f.append(date_filt)
        else:
            dates_f.append(current_date)
    data = {'Title': titles, 'Link': links, 'Date': dates_f}

  except Exception as e:
    print(f'Ошибка перевода даты {e}')
    data = {'Title': titles, 'Link': links, 'Date': dates}

  df_kun_uz = pd.DataFrame(data)

  return df_kun_uz


def sputniknews(queries):

  titles, links, dates = [], [], []

  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  chromedriver_autoinstaller.install()
  driver = webdriver.Chrome(options=chrome_options)
  driver.maximize_window()

  current_time = datetime.now()
  current_year = current_time.strftime("%Y")

  for query in queries:

      link_of_query = f'https://uz.sputniknews.ru/search/?query={query}'
      driver.get(link_of_query)
      wait = WebDriverWait(driver, 10)

      try:
          info_elements = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "list__content")))
          date_elements = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "list__date ")))
      except Exception as e:
          print(f'Произошла ошибка с поиском ифнормации- {e} по ссылке {link_of_query}')

      try:
        for single_news in info_elements:

          link_element = single_news.find_element(By.TAG_NAME, 'a')
          titles.append(single_news.text)
          links.append(link_element.get_attribute("href"))

        for single_news_date in date_elements:
          dates.append(single_news_date.text)
      except Exception as e:
          print(f'Произошла ошибка с - {e} по ссылке {link_of_query}')

  driver.quit()


  dates_f = []

  for date in dates:
      if ',' in date:
          date_filt = ' '.join(date.split(', ')[:-1])
          if len(date_filt.split()) == 2:
              dates_f.append(date_filt + ' ' + current_year)
          else:
              dates_f.append(date_filt)
      else:
          dates_f.append(' ')

  try:
      if len(titles) != len(links) or len(titles) != len(dates_f):
          raise ValueError("Длины заголовков, ссылок и дат должны быть равными.")

      data = {'Title': titles, 'Link': links, 'Date': dates_f}
      df_sputniknews = pd.DataFrame(data)

      return df_sputniknews

  except ValueError as ve:
      print("ValueError на сайте sputniknews:", ve)
      return None
  except Exception as e:
      print("An unexpected error на сайте sputniknews:", e)
      return None

def gazeta_uz(queries):
    titles, dates, links = [], [], []
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chromedriver_autoinstaller.install()
    driver = webdriver.Chrome(options=chrome_options)

    current_time = datetime.now()

    for query in queries:
        link = 'https://www.gazeta.uz/ru/search/?q=' + query
        driver.get(link)
        wait = WebDriverWait(driver, 10)

        try:
            info_pages = wait.until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'ut')))
            number_of_pages = 1
            if info_pages:
                number_of_pages = int(info_pages[0].text.strip()[-1])
        except Exception as e:
            print(f"Error retrieving number of pages for query '{query}': {str(e)}")

        for page in range(number_of_pages):
            info = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'nblock')))
            for news in info:
                date, title = news.text.split('\n')[:2]

                date = date.split()[:3]
                date_str = ' '.join(i.rstrip(',') for i in date)

                if  'Сегодня' in date_str:
                    dates.append(current_time.strftime("%d.%m.%Y"))
                elif 'Вчера' in date_str:
                    dates.append((current_time - timedelta(days=1)).strftime("%d.%m.%Y"))
                else:
                    dates.append(date_str)


                titles.append(title)
                link_element = news.find_element(By.CLASS_NAME, 'nimg')
                link = link_element.get_attribute("href")
                links.append(link)

            try:
                see_more_button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'next')))
                see_more_button.click()
            except Exception as e:
                print(f"Error clicking 'Next' button for query '{query}' on page {page + 1}: {str(e)}")
                break
    driver.quit()

    try:
        if len(titles) != len(links) or len(titles) != len(dates):
            raise ValueError("Длины заголовков, ссылок и дат должны быть равными.")

        data = {'Title': titles, 'Link': links, 'Date': dates}
        df_gazeta = pd.DataFrame(data)

        return df_gazeta

    except ValueError as ve:
        print("ValueError на сайте gazeta:", ve)
        return None
    except Exception as e:
        print("An unexpected error на сайте gazeta:", e)
        return None
def dzen():
  titles, links, dates = [], [], []
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  chromedriver_autoinstaller.install()
  driver = webdriver.Chrome(options=chrome_options)
  driver.maximize_window()

  current_time = datetime.now()

  query = 'https://dzen.ru/news/region/uzbekistan'
  driver.get(query)
  wait = WebDriverWait(driver, 10)
  try:
      title_elements = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "news-card2-redesign__title")))
      date_elements = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "news-story-redesign__time")))
      link_elements = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "Link")))
  except Exception as e:
      print('Произошла ошибка с ', e)

  try:
    for single_news_title in title_elements:

      titles.append(single_news_title.text)

    for single_news_date in date_elements:
      if 'назад'in single_news_date.text or 'Сегодня' in single_news_date.text:
          dates.append(current_time.strftime("%d.%m.%Y"))
      elif 'Вчера' in single_news_date.text:
          dates.append((current_time - timedelta(days=1)).strftime("%d.%m.%Y"))
      else:
          dates.append(f'{single_news_date.text} {current_time.strftime("%Y")}')
    for single_news_link in link_elements:
      links.append(single_news_link.get_attribute("href"))
  except Exception as e:
      print('Произошла ошибка с ', e)

  driver.quit()

  try:
      if len(titles) != len(links) or len(titles) != len(dates):
          raise ValueError("Длины заголовков, ссылок и дат должны быть равными.")

      data = {'Title': titles, 'Link': links, 'Date': dates}
      df_dzen = pd.DataFrame(data)
      return df_dzen

  except ValueError as ve:
      print("ValueError на сайте dzen:", ve)
      return None
  except Exception as e:
      print("An unexpected error на сайте dzen:", e)
      return None


def uzbekistonmet_uz(queries):

  titles, links, dates = [], [], []

  for query in queries:
      chrome_options = webdriver.ChromeOptions()
      chrome_options.add_argument('--headless')
      chrome_options.add_argument('--no-sandbox')
      chrome_options.add_argument('--disable-dev-shm-usage')
      chromedriver_autoinstaller.install()
      driver = webdriver.Chrome(options=chrome_options)
      driver.maximize_window()

      link = f'https://www.uzbekistonmet.uz/ru/search/index?SearchForm%5Btext%5D={query}'
      driver.get(link)
      wait = WebDriverWait(driver, 10)

      try:
          info_elements = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "title_link")))
          date_elements = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "date_time")))
      except Exception as e:
          print('Произошла ошибка с ', e)

      try:
        for single_news_info in info_elements:
          titles.append(single_news_info.text)
          links.append(single_news_info.get_attribute("href"))
        for single_news_date in date_elements:
          dates.append(single_news_date.text)
      except Exception as e:
          print('Произошла ошибка с ', e)

      driver.quit()


  titles_f, links_f, date_f = [], [], []

  dates_filt = [info.split(' | ') for info in dates]

  for title, link, date in zip(titles, links, dates_filt):
    if date[1] != 'Вакансии':
      titles_f.append(title)
      links_f.append(link)
      date_f.append(date[0])
    else:
      continue

  try:
      if len(titles) != len(links) or len(titles) != len(dates):
          raise ValueError("Длины заголовков, ссылок и дат должны быть равными.")

      data_f = {'Title': titles_f, 'Link': links_f, 'Date': date_f}
      df_uzbekistonmet_uz = pd.DataFrame(data_f)

      return df_uzbekistonmet_uz

  except ValueError as ve:
      print("ValueError на сайте uzbekistonmet:", ve)
      return None
  except Exception as e:
      print("An unexpected error на сайте uzbekistonmet:", e)
      return None



def bbgl_ru():

  links_of_queries = ['https://bbgl.ru/news?hashtag_place=%D0%A3%D0%B7%D0%B1%D0%B5%D0%BA%D0%B8%D1%81%D1%82%D0%B0%D0%BD',
          'https://bbgl.ru/obekty-stroitelstva?hashtag_place=%D0%A3%D0%B7%D0%B1%D0%B5%D0%BA%D0%B8%D1%81%D1%82%D0%B0%D0%BD']

  titles, links, dates = [], [], []

  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  chromedriver_autoinstaller.install()
  driver = webdriver.Chrome(options=chrome_options)
  driver.maximize_window()
  wait = WebDriverWait(driver, 10)

  for link in links_of_queries:
    driver.get(link)

    try:
        date_elements = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "date")))
        info_elements = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "title-link")))
    except Exception as e:
        print(f'Произошла ошибка с {link}', e)

    try:
      for single_news_info in info_elements:
        titles.append(single_news_info.text)
        links.append(single_news_info.get_attribute("href"))

      for single_news_info in date_elements:
        dates.append(single_news_info.text)

    except Exception as e:
        print(f'Произошла ошибка с {link}', e)

  driver.quit()

  dates_f = [' '.join(date.split()[:3]) for date in dates]

  try:
      if len(titles) != len(links) or len(titles) != len(dates_f):
          raise ValueError("Длины заголовков, ссылок и дат должны быть равными.")

      data = {'Title': titles, 'Link': links, 'Date': dates_f}
      df_bbgl = pd.DataFrame(data)
      return df_bbgl

  except ValueError as ve:
      print("ValueError на сайте bbgl.ru:", ve)
      return None
  except Exception as e:
      print("An unexpected error на сайте bbgl.ru:", e)
      return None

In [ ]:
def convert_date(date_str):
    if pd.isna(date_str):
        return None
    if len(date_str.split('.')) != 3:
        parts = date_str.split()
        if len(parts) == 3:
            months = {
                'января': 1,
                'февраля': 2,
                'марта': 3,
                'апреля': 4,
                'мая': 5,
                'июня': 6,
                'июля': 7,
                'августа': 8,
                'сентября': 9,
                'октября': 10,
                'ноября': 11,
                'декабря': 12
            }
            day, month, year = parts
            month = months.get(month.lower())
            if month:
                return f"{day}.{month:02d}.{year}"
            else:
                return None
        else:
            return None
    else:
        return date_str

In [ ]:
def news_to_excel(start_date=None, end_date=None, queries=None):

    if end_date is None:
        end_date = datetime.now().strftime('%Y-%m-%d')
    if start_date is None:
        start_date = (datetime.now() - timedelta(days=29)).strftime('%Y-%m-%d')
    if queries is None:
      queries = ['инвест', 'разраб', 'строит', 'модерн']

    df_kun_uz = kun_uz(queries)
    df_sputniknews = sputniknews(queries)
    df_gazeta_uz = gazeta_uz(queries)
    df_dzen = dzen()
    df_uzbekistonmet_uz = uzbekistonmet_uz(queries)
    df_bbgl = bbgl_ru()

    dfs = [df_kun_uz, df_sputniknews, df_gazeta_uz, df_dzen, df_uzbekistonmet_uz, df_bbgl]
    dfs_valid = []

    for i, df in enumerate(dfs):
        if not df.empty:
          dfs_valid.append(df)
        else:
          print(f'Ошибка с сайтом - {dfs[i]}')

    combined_df = pd.concat(dfs_valid, ignore_index=True)
    combined_df['Date'] = combined_df['Date'].apply(convert_date)
    combined_df['Date'] = pd.to_datetime(combined_df['Date'], format='%d.%m.%Y')

    end_date = datetime.now()
    start_date = end_date - timedelta(days=29)
    news_last_days = combined_df[(combined_df['Date'] >= start_date) & (combined_df['Date'] <= end_date)]
    news_last_days['Date'] = news_last_days['Date'].dt.strftime('%d-%m-%Y')
    return news_last_days

#Блок проектов

In [ ]:
def invest_in_uzbekistan():

  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  chromedriver_autoinstaller.install()
  driver = webdriver.Chrome(options=chrome_options)
  driver.maximize_window()
  titles, links, investments = [], [], []
  link = 'https://invest-in-uzbekistan.org/enterprises-projects/'
  driver.get(link)
  wait = WebDriverWait(driver, 10)
  try:
      price_elements = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "main-search-result-price")))
      title_elements = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "main-search-result-title")))
      link_elements = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="main-search-results"]/div/div/div/div[1]/a')))

  except Exception as e:
      print('Произошла ошибка с ', e)

  try:

    for price, title, link in zip(price_elements, title_elements, link_elements):
      titles.append(title.text)
      links.append(link.get_attribute("href"))

      invest_str = ''.join(filter(str.isdigit,(price.text.split(': ')[1])))

      if invest_str:
        investments.append(int(invest_str))

      else:
        investments.append(0)


  except Exception as e:
      print('Произошла ошибка с ', e)
  driver.quit()


  try:
      if len(titles) != len(links) or len(titles) != len(investments):
          raise ValueError("Длины заголовков, ссылок и дат должны быть равными.")

      data = {'Title': titles, 'Link': links, 'Investments ($)': investments}
      df_invest_in_uzbekistan = pd.DataFrame(data)
      return df_invest_in_uzbekistan

  except ValueError as ve:
      print("ValueError на сайте invest_in_uzbekistan:", ve)
      return None
  except Exception as e:
      print("An unexpected error на сайте invest_in_uzbekistan:", e)
      return None

def invest_gov_uz():

  titles, links, investments = [], [], []
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  chromedriver_autoinstaller.install()
  driver = webdriver.Chrome(options=chrome_options)


  link = 'https://invest.gov.uz/ru/investor-taxonomy/ready-invest-projects/'
  driver.get(link)

  wait = WebDriverWait(driver, 10)

  try:

      info = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'content-item-title')))
      invest_info = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'content-item-price')))

  except Exception as e:

      print(f"Ошибка извлечения информации {str(e)}")
  try:
    for single_news in info:
      link_element = single_news.find_element(By.TAG_NAME, 'a')
      titles.append(single_news.text)
      links.append(link_element.get_attribute("href"))

    for single_inv in invest_info:

      invest_str = ''.join(filter(str.isdigit, single_inv.text))
      if invest_str:
        investments.append(int(invest_str))
      else:
        investments.append(0)

  except Exception as e:
    print('Ошибка - ', e)
  driver.quit()

  try:
      if len(titles) != len(links) or len(titles) != len(investments):
          raise ValueError("Длины заголовков, ссылок и дат должны быть равными.")

      data = {'Title': titles, 'Link': links, 'Investments ($)': investments}
      df_invest_qov_uz = pd.DataFrame(data)
      return df_invest_qov_uz

  except ValueError as ve:
      print("ValueError на сайте invest_gov_uz:", ve)
      return None
  except Exception as e:
      print("An unexpected error на сайте invest_gov_uz:", e)
      return None

In [ ]:
def inv_projects_to_excel():


    df_invest_qov_uz = invest_gov_uz()
    df_invest_in_uzbekistan = invest_in_uzbekistan()


    dfs = [df_invest_qov_uz, df_invest_in_uzbekistan]
    dfs_valid = []

    for i, df in enumerate(dfs):
        if not df.empty:
          dfs_valid.append(df)
        else:
          print(f'Ошибка с сайтом - {dfs[i]}')

    df_inv = pd.concat(dfs_valid, ignore_index=True)

    return df_inv

# Вывод общей excel таблицы

In [ ]:
with pd.ExcelWriter(f"news_inv_projects_{datetime.now().strftime('%B_%Y')}.xlsx", engine='xlsxwriter') as writer:
    news_to_excel().to_excel(writer, sheet_name=f'news', index=False)
    inv_projects_to_excel().to_excel(writer, sheet_name='invest_projects', index=False)